In [6]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [37]:
documents = []
for i in range(0,50):
    try:
        with open('data/video' + str(i+1) + '.txt', 'r') as myfile:
            data=myfile.read().replace('\n', '')
        data = data.replace('%HESITATION', '')
        documents.append(data)
    except:
        pass

In [38]:
print(len(documents))

48


In [40]:
documents

["resolution my name's Bobby does show an injured around we're going to sit down with twenty five masters of the design industry and we're gonna learn from them we're gonna learn how the best companies in the world approach communicate and deployed design in their businesses every single day in this episode we're talking with Katie deal Katie is the director of experience design and Aaron be should focus on how to frame user experiences and you storytelling to lead a discussion  we're gonna get to that stick around right after these messages  thanks for the support whether you need a domain a website or an online store make your next move with squarespace visits where space not calm and into the code high resolution one word for ten percent off your first purchase  thanks for joining us thanks for having me  what's one thing about designed it's clear to you that you feel is not so clear other people in  it's getting better but I would say  some still don't know that design is more than

In [41]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(documents)

In [42]:
true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=2, n_init=1, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [45]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

print("\n")
print("Prediction")

Y = vectorizer.transform(["chrome browser to open."])
prediction = model.predict(Y)
print(prediction)

Y = vectorizer.transform(["My cat is hungry."])
prediction = model.predict(Y)
print(prediction)

Top terms per cluster:
Cluster 0:
 like
 know
 people
 just
 think
 design
 right
 really
 things
 yeah
Cluster 1:
 blockchain
 people
 want
 space
 just
 kind
 lot
 start
 meet
 educate


Prediction
[0]
[0]


In [49]:
from sklearn.feature_extraction.text import CountVectorizer
from stop_words import get_stop_words
# use a custom stopwords list, set the minimum term-document frequency to 20
vectorizer = CountVectorizer(stop_words = get_stop_words('en'), min_df = 20)
A = vectorizer.fit_transform(documents)
print( "Created %d X %d document-term matrix" % (A.shape[0], A.shape[1]) )

Created 48 X 389 document-term matrix


/home/akul/Desktop/venv/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'can', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [50]:
terms = vectorizer.get_feature_names()
print("Vocabulary has %d distinct terms" % len(terms))

Vocabulary has 389 distinct terms


In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
# we can pass in the same preprocessing parameters
vectorizer = TfidfVectorizer(stop_words=get_stop_words('en'), min_df = 20)
A = vectorizer.fit_transform(documents)
print( "Created %d X %d TF-IDF-normalized document-term matrix" % (A.shape[0], A.shape[1]) )

Created 48 X 389 TF-IDF-normalized document-term matrix


In [55]:
# extract the resulting vocabulary
terms = vectorizer.get_feature_names()
print("Vocabulary has %d distinct terms" % len(terms))

Vocabulary has 389 distinct terms


In [56]:
import operator
def rank_terms( A, terms ):
    # get the sums over each column
    sums = A.sum(axis=0)
    # map weights to the terms
    weights = {}
    for col, term in enumerate(terms):
        weights[term] = sums[0,col]
    # rank the terms by their weight over all documents
    return sorted(weights.items(), key=operator.itemgetter(1), reverse=True)

In [57]:
ranking = rank_terms( A, terms )
for i, pair in enumerate( ranking[0:20] ):
    print( "%02d. %s (%.2f)" % ( i+1, pair[0], pair[1] ) )

01. like (13.26)
02. know (10.10)
03. re (7.47)
04. can (7.40)
05. people (7.24)
06. just (6.67)
07. think (6.47)
08. design (5.81)
09. really (5.61)
10. right (5.54)
11. one (5.06)
12. get (4.53)
13. things (4.48)
14. yeah (4.40)
15. want (4.15)
16. kind (3.90)
17. product (3.64)
18. time (3.50)
19. actually (3.44)
20. don (3.38)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import NMF, LatentDirichletAllocation

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

no_topics = 20

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

In [ ]:
def cluster_texts(texts, clusters=3):
""" Transform texts to Tf-Idf coordinates and cluster texts using K-Means """
vectorizer = TfidfVectorizer(tokenizer=process_text,
stop_words=stopwords.words('english'),
max_df=0.5,
min_df=0.1,
lowercase=True)
tfidf_model = vectorizer.fit_transform(texts)
km_model = KMeans(n_clusters=clusters)
km_model.fit(tfidf_model)
clustering = collections.defaultdict(list)
for idx, label in enumerate(km_model.labels_):
clustering[label].append(idx)
return clustering
if __name__ == "__main__":
articles = [...]
clusters = cluster_texts(articles, 7)
pprint(dict(clusters))
